In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# AI Agent - Orchestration with LLM

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [3]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Import libraries

In [4]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding

### Initalize Vertex AI

In [5]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

In [7]:
# @title Helper function
from vertexai.generative_models import ChatSession, GenerationConfig

def interactive_chat(chat: ChatSession, prompt: str, search:bool) -> str:

    tool = None
    if search:
      tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40
    }

    tools = [tool] if search else None

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings=safety_settings,
        tools = tools,
        stream=False
    )
    return responses.text

#------------------------------------------------------------------------------------

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  return history

#------------------------------------------------------------------------------------
def query_classification(chat_history, query):

  prompt = f"""당신은 이전 대화 내용을  참조하여 <입력질문>을 분류해주는 AI Assistant 입니다.
  1. 질문 분류를 기준은 아래 <분류기준>을 참고하여 아래 내용을 포함된 정보로 분류해주세요.
     - 분류이유
     - 분류결과
     - Endpoint

  2. 질문내의 키워드를 추출해서 정리해주세요.

  <대화내용>{chat_history}</대화내용>
  <입력질문>{query}</입력질문>

  <질문분류기준> [분류기준:endpoint]
  1. 영화추천 : http://media.com/영화추천
  2. 음악추천 : http://media.com/음악추천
  3. 기타 : http://media.com/기타

  """

  response_schema = {
      "type": "OBJECT",
      "properties": {
        "reason": {"type": "STRING"},
        "classification": {
            "type": "OBJECT",
            "properties": {
              "class": {"type": "STRING"},
              "endpoint": {"type": "STRING"},
            },
        }
      }
  }

  generation_config = GenerationConfig(
        temperature=0,
        top_p=1.0,
        top_k=10,
        candidate_count=1,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema

    )

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

#------------------------------------------------------------------------------------

def agent_1(chat_history, query):

  prompt =f"""
          당신은 영화정보를 제공하는AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 영화 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
          2. 영화를 선정 하는 이유를 설명 해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}
  """
  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],

  )

  return responses.text

def agent_2(chat_history, query):

  prompt =f"""
          당신은 음악정보를 제공하는 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 오늘 날씨에 맞는 음악을 추천해주세요.
          2. 추천하는 이유를 자세히 설명해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text

def agent_3(chat_history, query):

  prompt =f"""
          당신은 일상적인 대화가 가능한 AI Agent 입니다.
          가벼운 대화를 진행하되 혹시 질문이 있을 경우 해당 질문에 대해서 답변해주세요.
          전체 답변은 10줄을 넘지 않게 해주세요.
          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text


def agent_orchestration(chat_history, query):

  classified = query_classification(chat_history, query)
  task = eval(classified)['classification']['class']

  print(f"Agent Task : {task}")

  if task == '영화추천':
    return agent_1(chat_history, query)
  elif task == '음악추천':
    return agent_2(chat_history, query)
  else:
    return agent_3(chat_history, query)


In [8]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  chat_history = get_chat_history(chat)

  response = agent_orchestration(chat_history, query)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 오늘 날씨 검색해서 음악 추천해주세요.
Agent Task : 음악추천


AI Agent : 오늘(12월 27일) 서울 날씨에 맞는 음악 추천입니다.  제공된 정보에 따르면 오늘 서울 날씨는 오전에는 맑음, 오후에는 구름이 많고 강수 확률이 30%입니다.  기온은 영하에서 영상 1도 사이로 예상됩니다.

따라서 추천하는 음악은 다음과 같습니다:

**1. 잔잔한 어쿠스틱 기타 연주곡:**  오후에 구름이 많고 쌀쌀한 날씨를 고려하여, 차분하고 따뜻한 느낌을 주는 어쿠스틱 기타 음악이 적절합니다.  어쿠스틱 기타의 포근한 선율은 겨울의 차가운 날씨를 부드럽게 감싸주는 듯한 효과를 줍니다.  비가 오지 않더라도, 흐린 날씨의 쓸쓸함을 달래주는 데 효과적입니다.

**2. 겨울 분위기의 재즈:** 차분하면서도 세련된 재즈 음악은 겨울의 특유한 정취를 느끼게 해줍니다.  피아노의 섬세한 건반 소리와 잔잔한 색소폰 연주는 겨울의 차가운 공기를 따뜻하게 만들어줍니다.  강수 확률이 있으므로, 실내에서 듣기에 좋습니다.

**3.  가볍고 경쾌한 팝송:**  맑은 오전 날씨에는 가볍고 경쾌한 팝송이 기분을 좋게 해줍니다.  활기찬 멜로디는 잠자던 기분을 깨워주고 하루를 긍정적으로 시작하게 해줍니다.  오후의 흐린 날씨에도, 밝은 분위기의 음악은 우울함을 날려버리는 데 도움이 됩니다.


이 음악들은 날씨의 다양한 측면 (맑음, 구름, 기온, 강수 확률)을 고려하여 선택되었으며,  듣는 사람의 감정과 분위기에 맞춰 선택할 수 있습니다.  오늘 하루 즐거운 음악 감상이 되시길 바랍니다.


------------------------------------ 
사용자: 오늘 날씨에 맞는 영화 추천해주세요.
Agent Task : 영화추천


AI Agent : 오늘 날씨에 맞는 영화 추천입니다.

1. 비가 온다면, 박찬욱 감독의  <헤어질 결심>을 추천합니다. 빗속의 쓸쓸하고 아련한 분위기와 영화의 분위기가 잘 어울립니다.  또한, 섬세한 연출과 배우들의 연기가 감탄스럽습니다.

2.  만약 맑은 날이라면,  <리틀 포레스트>를 추천합니다.  따뜻하고 아름다운 시골 풍경과 소소한 일상이 힐링을 선사할 것입니다.  다양한 계절의 아름다움을 느낄 수 있어 기분 전환에 좋습니다.


선정 이유:

<헤어질 결심>은 비 오는 날의 쓸쓸하고 감성적인 분위기와 잘 어울리는 멜로 영화입니다.  <리틀 포레스트>는 맑은 날의 편안하고 긍정적인 분위기에 맞춰 힐링과 따스함을 주는 영화입니다. 날씨에 따라 영화의 분위기를 고려하여  선택하였습니다.  두 영화 모두 뛰어난 영상미와 스토리텔링을 가지고 있습니다.


------------------------------------ 
사용자: 날씨를 검색해서 영화 추천해주세요.
Agent Task : 영화추천


AI Agent : 1. 날씨 정보가 없으므로 일반적인 겨울 날씨에 어울리는 영화를 추천합니다.  "러브 액츄얼리", "해리포터" 시리즈, "겨울왕국"을 추천합니다.  따뜻한 감성과 판타지, 아름다운 영상미가 겨울 분위기와 잘 어울립니다.

2.  "러브 액츄얼리"는 다양한 사랑 이야기로 따뜻함을 선사하고, "해리포터" 시리즈는  겨울 배경의 판타지 세계로 흥미를 더하며, "겨울왕국"은 아름다운 영상미와 감동적인 스토리로 겨울 분위기를 한껏 고조시키기 때문입니다.

3. 추운 날씨에는 따뜻하고 감동적인 영화가 제격입니다.  위 세 영화는 각각 로맨스, 판타지, 애니메이션 장르로 다양한 취향을 만족시킵니다.  "러브 액츄얼리"의 따뜻한 유머와 감동, "해리포터" 시리즈의 마법과 모험, "겨울왕국"의 아름다운 음악과 스토리는 추운 날씨에도 마음을 훈훈하게 해줄 것입니다.  집에서 편안하게 감상하기에 좋은 영화들입니다.  즐거운 관람 되세요!


------------------------------------ 
사용자: 오늘 날씨 검색해서 영화 추천해주세요.
Agent Task : 영화추천


AI Agent : 오늘(12월 27일) 날씨를 고려하여 영화를 추천해 드리겠습니다.

1. 추천 영화:  "홈 얼론" (코미디), "러브 액츄얼리" (로맨틱 코미디)

2. 이유: 오늘 날씨가 춥고,  "홈 얼론"은 따뜻한 실내에서 즐길 수 있는 유쾌한 코미디 영화이며, "러브 액츄얼리"는 따뜻하고 포근한 분위기의 로맨틱 코미디로 추운 날씨에 어울리는 영화입니다.


3. 선정 이유 상세 설명:

오늘 날씨는 대체로 춥고, 강수 확률이 있습니다.  따라서 실내에서 편안하게 즐길 수 있는 영화가 좋겠습니다. "홈 얼론"은 가족과 함께 즐거운 시간을 보낼 수 있는 코미디 영화이고, "러브 액츄얼리"는 연인 또는 친구와 함께 감상하기 좋은 로맨틱 코미디 영화입니다.  추운 날씨에 따뜻한 마음을 전달해 줄 수 있는 영화들입니다.


------------------------------------ 
사용자: 종료
